In [2]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Load dataset
data = pd.read_csv('turf_price_dataset.csv')  # Replace with your actual CSV file

# Encode categorical variables
label_encoders = {}
categorical_cols = ['Day Time', 'Season', 'Time Slot', 'Weather', 'Day Type', 'Special Event', 'Turf Quality']

for col in categorical_cols:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le  # Store label encoders for later use

# Define features and target
X = data.drop(columns=['Price Per Hour'])
y = data['Price Per Hour']

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate model
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error: {mae}')

# Save model and encoders
joblib.dump(model, 'price_prediction_model.pkl')
joblib.dump(label_encoders, 'label_encoders.pkl')

# Function to predict price per hour
def predict_price():
    # Load model and encoders
    model = joblib.load('price_prediction_model.pkl')
    label_encoders = joblib.load('label_encoders.pkl')
    
    # Take user input
    user_input = {}
    user_input['Day Time'] = input("Enter Day Time (Evening, Afternoon, Morning, Night): ")
    user_input['Season'] = input("Enter Season (Off-peak, Peak): ")
    user_input['Time Slot'] = input("Enter Time Slot (e.g., '9 AM - 12 PM', '12 PM - 3 PM'): ")
    user_input['Weather'] = input("Enter Weather (Clear, Stormy, Rainy, Cloudy): ")
    user_input['Day Type'] = input("Enter Day Type (Weekday, Weekend): ")
    user_input['Special Event'] = input("Is there a Special Event? (Yes, No): ")
    user_input['Turf Quality'] = input("Enter Turf Quality (Standard, Basic, Premium): ")
    user_input['Location Demand'] = int(input("Enter Location Demand (1-10): "))
    
    # Encode categorical values
    for col in categorical_cols:
        if user_input[col] in label_encoders[col].classes_:
            user_input[col] = label_encoders[col].transform([user_input[col]])[0]
        else:
            print(f"Invalid input for {col}. Exiting.")
            return
    
    # Convert to DataFrame
    input_df = pd.DataFrame([user_input])
    
    # Predict price
    predicted_price = model.predict(input_df)[0]
    print(f'Predicted Price Per Hour: {predicted_price}')

# Run manual input function
if __name__ == '__main__':
    predict_price()

Mean Absolute Error: 49.52
Predicted Price Per Hour: 1081.0
